In [21]:
import pandas as pd
import numpy as np
import geopandas as gpd

from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_image
from functools import partial
from rasterio.enums import MergeAlg
from rasterio.mask import mask

import rasterio


## Rasterize canopy area from the individual tree point shapefiles

### Load borough tree crown data

In [2]:
### Load data
x = gpd.read_file("../data/greenness/TreePoint/BronxP.shp")
x1 = gpd.read_file("../data/greenness/TreePoint/BrooklynP.shp")
x2 = gpd.read_file("../data/greenness/TreePoint/QueensP.shp")
x3 = gpd.read_file("../data/greenness/TreePoint/ManhP.shp")
x4 = gpd.read_file("../data/greenness/TreePoint/StateIP.shp")

### Append the borough tree crown data

In [3]:
# Append all the borough datasets together
nyc_trees = pd.concat([x, x1, x2, x3, x4])

### Convert the tree crown polygons to raster datasets using the 'Area' attribute

-  Canopy cover is a measure of the percentage of ground covered by a vertical projection of the tree canopy.
- Here we are summing the crown areas (m^2) within each hectare (100x100m) grid cell.
- To get canopy cover as a percentage, we simply divide each cell in teh resulting array by 10,000m^2

In [30]:
# Points to raster (summing crown area)

nyc_trees.crs # already in a a metered projection so we're all set

crown_raster = make_geocube(vector_data=nyc_trees,
                            measurements = ['Area'],
                            resolution = (-100,100),
                            rasterize_function = partial(rasterize_image, merge_alg = MergeAlg.add),
                            fill = 0)


In [31]:
(crown_raster["Area"]/10000)*100

<xarray.DataArray 'Area' (y: 469, x: 468)>
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])
Coordinates:
  * y            (y) float64 4.53e+06 4.53e+06 4.53e+06 ... 4.483e+06 4.483e+06
  * x            (x) float64 5.63e+05 5.632e+05 ... 6.096e+05 6.098e+05
    spatial_ref  int64 0

In [32]:
crown_raster['Area'] = (crown_raster.Area/10000)*100

In [33]:
# Save crown area raster
crown_raster.rio.to_raster('../data/L1/crown_area_ha.tiff')

xarray.core.dataset.Dataset

## Rasterize the individual tree data to get tree density per hectare (100mx100m)

In [42]:
nyc_trees['value'] = 1 # this is so that we can calculate density in the rasterization step

nyc_trees

,ID,VALUE,NAME,Area,RG,ORIG_FID,geometry,FID_,X,value
0,29.0,50.0,50,2,1.0,0,POINT (594802.000 4515982.480),NaN,NaN,1
1,29.0,50.0,50,60,1.0,1,POINT (593945.550 4516008.147),NaN,NaN,1
2,29.0,50.0,50,3,1.0,2,POINT (594814.500 4516025.480),NaN,NaN,1
3,29.0,50.0,50,21,1.0,3,POINT (593879.738 4516080.432),NaN,NaN,1
4,29.0,50.0,50,9,1.0,4,POINT (594997.500 4516100.480),NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...
1744357,229.0,250.0,250,57,NaN,1744357,POINT (575854.750 4498382.430),NaN,NaN,1
1744358,229.0,250.0,250,52,NaN,1744358,POINT (575854.500 4498377.543),NaN,NaN,1
1744359,229.0,250.0,250,59,NaN,1744359,POINT (575861.381 4498371.700),NaN,NaN,1
1744360,219.0,240.0,240,97,NaN,1744360,POINT (579674.710 4495625.600),NaN,NaN,1


In [47]:
# Create point density (sum of trees per 100m^2 grid cell)

tree_density = make_geocube(vector_data = nyc_trees,
                           measurements = ['value'],
                           resolution = (-100,100), # hectare resolution
                           rasterize_function = partial(rasterize_image, merge_alg = MergeAlg.add),
                           fill = 0) 

In [49]:
# Save raster census raster
tree_density.rio.to_raster('../data/L1/tree_counts_per_ha.tiff')

In [48]:
tree_density.value

<xarray.DataArray 'value' (y: 469, x: 468)>
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])
Coordinates:
  * y            (y) float64 4.53e+06 4.53e+06 4.53e+06 ... 4.483e+06 4.483e+06
  * x            (x) float64 5.63e+05 5.632e+05 ... 6.096e+05 6.098e+05
    spatial_ref  int64 0
Attributes:
    name:        value
    long_name:   value
    _FillValue:  0